In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Nos movemos a /content
%cd /content


# Descomprimir (añade -q para que no saque tanto texto si quieres)
!unzip "/content/drive/MyDrive/IA_Image_Detection/IA-Image-Detection_v2.zip"


In [3]:
#CELDA 5- verificar que todo se ha descomprimido bien
print("Contenido de /content:")
!ls /content

print("\nContenido de /content/IA-Image-Detection:")
!ls /content/IA-Image-Detection

print("\nSubcarpetas dentro de IA-Image-Detection:")
!ls /content/IA-Image-Detection


Contenido de /content:
drive  IA-Image-Detection  sample_data

Contenido de /content/IA-Image-Detection:
config.yaml  models	README.md	  results  src
data	     notebooks	requirements.txt  scripts  venv

Subcarpetas dentro de IA-Image-Detection:
config.yaml  models	README.md	  results  src
data	     notebooks	requirements.txt  scripts  venv


In [4]:
#CELDA 6 - Instalar dependencias del proyecto
!pip install -q timm tqdm scikit-learn opencv-python pillow
print("Dependencias instaladas.")


Dependencias instaladas.


In [15]:
#CELDA 7 - variables del configuración (fáciles de manejar)
PROJECT_DIR = "/content/IA-Image-Detection"
SRC_DIR     = f"{PROJECT_DIR}/src"
DATA_DIR    = f"{PROJECT_DIR}/data/raw"   # CAMBIA 'ADM' si usas otro dataset
SAVE_DIR    = f"{PROJECT_DIR}/models/saved_models"

print("PROJECT_DIR:", PROJECT_DIR)
print("SRC_DIR    :", SRC_DIR)
print("DATA_DIR   :", DATA_DIR)
print("SAVE_DIR   :", SAVE_DIR)


PROJECT_DIR: /content/IA-Image-Detection
SRC_DIR    : /content/IA-Image-Detection/src
DATA_DIR   : /content/IA-Image-Detection/data/raw
SAVE_DIR   : /content/IA-Image-Detection/models/saved_models


In [17]:
#CELDA 8 - crear carpetas de modelos si no existe
import os

os.makedirs(SAVE_DIR, exist_ok=True)
print("Carpeta de modelos asegurada:", SAVE_DIR)

%cd /content/IA-Image-Detection

!mkdir -p models/saved_models



Carpeta de modelos asegurada: /content/IA-Image-Detection/models/saved_models
/content/IA-Image-Detection


In [ ]:
#CELDA 9 - Entrenamiento simple (un único experimento)
# Entrenamiento simple (ajusta epochs, batch_size, lr a tu gusto)
%cd /content/IA-Image-Detection

!python src/train.py \
    --data_dir "/content/IA-Image-Detection/data/raw" \
    --epochs 6 \
    --batch_size 128 \
    --lr 8e-4



In [ ]:
#CELDA AUX - ver cuantas imagenes hay en cada carpeta
import os

base = "/content/IA-Image-Detection/data/raw"

for gen in os.listdir(base):
    gen_path = os.path.join(base, gen)
    if os.path.isdir(gen_path):
        print("\n=== GENERADOR:", gen, "===")
        for split in ["train", "val"]:
            for cls in ["ai", "nature"]:
                cls_path = os.path.join(gen_path, split, cls)
                if os.path.isdir(cls_path):
                    count = len([f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg','.png','.jpeg'))])
                    print(f"{split}/{cls}: {count} imágenes")


In [ ]:
#CELDA AUX - ver los 5 primeros archivos de cada carpeta
import os

base = "/content/IA-Image-Detection/data/raw"

for gen in os.listdir(base):
    gen_path = os.path.join(base, gen)
    if os.path.isdir(gen_path):
        print("\n=== GENERADOR:", gen, "===")
        for split in ["train", "val"]:
            for cls in ["ai", "nature"]:
                cls_path = os.path.join(gen_path, split, cls)
                if os.path.isdir(cls_path):
                    files = os.listdir(cls_path)[:5]
                    print(f"{split}/{cls}: {files}")


In [22]:
#CELDA AUX - crear carpeta data/raw/ALL_full
%%writefile scripts/create_full_all.py
import shutil
from pathlib import Path

# Carpeta donde tienes todos los generadores (BigGAN, GLIDE, etc.)
RAW_ROOT = Path("data/raw")

# Carpeta destino donde se creará el dataset combinado
DEST_ROOT = RAW_ROOT / "ALL_full"

# Carpetas a ignorar dentro de data/raw
IGNORE_DIRS = {"ALL_full", "_imagenes_corruptas"}

# Extensiones de imagen que vamos a copiar
EXTS = [".jpg", ".jpeg", ".png", ".bmp", ".webp",
        ".JPG", ".JPEG", ".PNG", ".BMP", ".WEBP"]


def get_all_images(folder: Path):
    """Devuelve una lista con todas las imágenes (recursivo) dentro de folder."""
    files = []
    for ext in EXTS:
        files.extend(folder.rglob(f"*{ext}"))
    return files


def process_generator(gen_dir: Path):
    gen_name = gen_dir.name
    print(f"\n=== Procesando generador: {gen_name} ===")

    for split in ["train", "val"]:
        for cls in ["ai", "nature"]:
            src = gen_dir / split / cls
            if not src.is_dir():
                print(f"[AVISO] Carpeta no encontrada: {src}")
                continue

            imgs = get_all_images(src)
            if len(imgs) == 0:
                print(f"[AVISO] No hay imágenes en {src}")
                continue

            dst = DEST_ROOT / split / cls
            dst.mkdir(parents=True, exist_ok=True)

            print(f"📂 Copiando {len(imgs)} imágenes de {src} -> {dst}")
            for img in imgs:
                # Prefijo con el nombre del generador para evitar colisiones
                dst_path = dst / f"{gen_name}_{img.name}"
                shutil.copy2(img, dst_path)

    print(f"=== Fin {gen_name} ===")


def main():
    DEST_ROOT.mkdir(parents=True, exist_ok=True)

    # Detectar generadores automáticamente (todas las carpetas en raw excepto las ignoradas)
    generators = [
        d for d in RAW_ROOT.iterdir()
        if d.is_dir() and d.name not in IGNORE_DIRS
    ]

    print("Generadores detectados:", [g.name for g in generators])

    for gen_dir in generators:
        process_generator(gen_dir)

    print("\n✅ Dataset combinado COMPLETO creado en:", DEST_ROOT)
    print("   Estructura: ALL_full/train/{ai,nature}, ALL_full/val/{ai,nature}")


if __name__ == "__main__":
    main()


Writing scripts/create_full_all.py


In [23]:
#CELDA AUX - ejecutar celda anterior
!python scripts/create_full_all.py


Generadores detectados: ['generador3_glide_descomprimido', 'generador2_BigGAN_descomprimido']

=== Procesando generador: generador3_glide_descomprimido ===
📂 Copiando 161999 imágenes de data/raw/generador3_glide_descomprimido/train/ai -> data/raw/ALL_full/train/ai
📂 Copiando 162000 imágenes de data/raw/generador3_glide_descomprimido/train/nature -> data/raw/ALL_full/train/nature
📂 Copiando 6000 imágenes de data/raw/generador3_glide_descomprimido/val/ai -> data/raw/ALL_full/val/ai
📂 Copiando 6000 imágenes de data/raw/generador3_glide_descomprimido/val/nature -> data/raw/ALL_full/val/nature
=== Fin generador3_glide_descomprimido ===

=== Procesando generador: generador2_BigGAN_descomprimido ===
📂 Copiando 161996 imágenes de data/raw/generador2_BigGAN_descomprimido/train/ai -> data/raw/ALL_full/train/ai
📂 Copiando 162000 imágenes de data/raw/generador2_BigGAN_descomprimido/train/nature -> data/raw/ALL_full/train/nature
📂 Copiando 6000 imágenes de data/raw/generador2_BigGAN_descomprimido/v

In [25]:
#CELDA 9_v2 - entrenamiento
%cd /content/IA-Image-Detection

!python src/train.py \
    --data_dir "/content/IA-Image-Detection/data/raw/ALL_full" \
    --epochs 10 \
    --batch_size 128 \
    --lr 8e-4


/content/IA-Image-Detection
Usando dataset: /content/IA-Image-Detection/data/raw/ALL_full
Dispositivo: cuda
Cargando datos desde los siguientes roots:
 - /content/IA-Image-Detection/data/raw/ALL_full
Clases detectadas: ['ai', 'nature']
Nº de datasets combinados: 1

--- Época 1/10 ---
Entrenando: 100% 5063/5063 [31:53<00:00,  2.65it/s]
Validando: 100% 188/188 [00:33<00:00,  5.66it/s]
Train Loss: 0.4528 | Val Loss: 0.0825 | Val Acc: 0.9767
Checkpoint guardado en models/saved_models/epoch_1.pth

--- Época 2/10 ---
Entrenando: 100% 5063/5063 [31:44<00:00,  2.66it/s]
Validando: 100% 188/188 [00:34<00:00,  5.37it/s]
Train Loss: 0.0977 | Val Loss: 0.0515 | Val Acc: 0.9845
Checkpoint guardado en models/saved_models/epoch_2.pth

--- Época 3/10 ---
Entrenando: 100% 5063/5063 [31:44<00:00,  2.66it/s]
Validando: 100% 188/188 [00:30<00:00,  6.22it/s]
Train Loss: 0.0819 | Val Loss: 0.0496 | Val Acc: 0.9869
Checkpoint guardado en models/saved_models/epoch_3.pth

--- Época 4/10 ---
Entrenando: 100% 50

In [13]:
#CELDA AUX - editar dataset.py (aumentar la velocidad)
from pathlib import Path

dataset_path = Path("/content/IA-Image-Detection/src/dataset.py")
print("Editando:", dataset_path)

text = dataset_path.read_text()

old_block = """    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    return train_loader, val_loader, class_names
"""

new_block = """    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        persistent_workers=True,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=max(4, num_workers // 2),
        pin_memory=True,
        persistent_workers=True,
    )

    return train_loader, val_loader, class_names
"""

if old_block not in text:
    raise RuntimeError("No he encontrado el bloque original en dataset.py. Revisa que el archivo es el correcto.")

text = text.replace(old_block, new_block)
dataset_path.write_text(text)

print("✅ dataset.py actualizado con pin_memory=True, persistent_workers=True y más num_workers para val_loader.")


Editando: /content/IA-Image-Detection/src/dataset.py
✅ dataset.py actualizado con pin_memory=True, persistent_workers=True y más num_workers para val_loader.


In [14]:
#CELDA AUX - guardar cambios del dataset.py en drive
!cp /content/IA-Image-Detection/src/dataset.py \
    "/content/drive/MyDrive/IA_Image_Detection/IA-Image-Detection/src/dataset.py"

print("✅ Copiado dataset.py modificado a tu carpeta del proyecto en Drive.")


cp: cannot create regular file '/content/drive/MyDrive/IA_Image_Detection/IA-Image-Detection/src/dataset.py': No such file or directory
✅ Copiado dataset.py modificado a tu carpeta del proyecto en Drive.


In [7]:
#CELDA AUX - detectar y mover imágenes corruptas
from pathlib import Path
from PIL import Image, UnidentifiedImageError
import shutil

DATA_ROOT = Path("/content/IA-Image-Detection/data/raw")  # raíz de todos tus generadores
BAD_ROOT = DATA_ROOT / "_imagenes_corruptas"
BAD_ROOT.mkdir(exist_ok=True)

total = 0
corruptas = 0

# Vamos a revisar PNG/JPG/JPEG por si acaso
extensiones = ("*.png", "*.jpg", "*.jpeg", "*.webp")

for patron in extensiones:
    for path in DATA_ROOT.rglob(patron):
        total += 1
        try:
            with Image.open(path) as img:
                img.verify()  # comprueba que el archivo es una imagen válida
        except (UnidentifiedImageError, OSError) as e:
            corruptas += 1
            print(f"[CORRUPTA] {path} -> {e}")
            # Movemos la imagen corrupta a otra carpeta para que el DataLoader no la vea
            destino = BAD_ROOT / path.name
            shutil.move(str(path), str(destino))

print(f"\nImágenes revisadas: {total}")
print(f"Imágenes corruptas movidas a {BAD_ROOT}: {corruptas}")


[CORRUPTA] /content/IA-Image-Detection/data/raw/generador3_glide_descomprimido/train/ai/GLIDE_1000_200_08_808_glide_00033.png -> cannot identify image file '/content/IA-Image-Detection/data/raw/generador3_glide_descomprimido/train/ai/GLIDE_1000_200_08_808_glide_00033.png'
[CORRUPTA] /content/IA-Image-Detection/data/raw/generador2_BigGAN_descomprimido/train/ai/116_biggan_00098.png -> cannot identify image file '/content/IA-Image-Detection/data/raw/generador2_BigGAN_descomprimido/train/ai/116_biggan_00098.png'
[CORRUPTA] /content/IA-Image-Detection/data/raw/generador2_BigGAN_descomprimido/train/ai/116_biggan_00094.png -> cannot identify image file '/content/IA-Image-Detection/data/raw/generador2_BigGAN_descomprimido/train/ai/116_biggan_00094.png'
[CORRUPTA] /content/IA-Image-Detection/data/raw/generador2_BigGAN_descomprimido/train/ai/116_biggan_00107.png -> cannot identify image file '/content/IA-Image-Detection/data/raw/generador2_BigGAN_descomprimido/train/ai/116_biggan_00107.png'
[COR

In [26]:
#CELDA 10 - guardar el mejor modelo
%cd /content/IA-Image-Detection

# Ejemplo: escogemos epoch_7
!cp models/saved_models/epoch_7.pth models/saved_models/best_model.pth


/content/IA-Image-Detection


In [27]:
#CELDA 11 - editar evaluate.py
%%writefile src/evaluate.py
import argparse
import torch
from sklearn.metrics import classification_report, confusion_matrix
from dataset import load_dataloaders
from model import build_model, load_checkpoint


def evaluate_model(model_path, data_dir, batch_size=64):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🖥️ Usando dispositivo: {device}")

    # SOLO cargamos el val_loader (no necesitamos train)
    _, val_loader, class_names = load_dataloaders(
        base_dir=data_dir,
        batch_size=batch_size,
    )

    # Crear modelo y cargar pesos
    model = build_model(num_classes=len(class_names), pretrained=False).to(device)
    model = load_checkpoint(model, path=model_path, device=device)
    model.eval()

    y_true, y_pred = [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    print("\n📊 === CLASIFICATION REPORT ===\n")
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

    print("\n📉 === MATRIZ DE CONFUSIÓN ===\n")
    print(confusion_matrix(y_true, y_pred))


def parse_args():
    parser = argparse.ArgumentParser(description="Evaluación del modelo ConvNeXt IA Detector")

    parser.add_argument("--model_path", type=str, required=True,
                        help="Ruta al archivo .pth del modelo")
    parser.add_argument("--data_dir", type=str, required=True,
                        help="Directorio base con train/ y val/")
    parser.add_argument("--batch_size", type=int, default=64)

    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()
    evaluate_model(
        model_path=args.model_path,
        data_dir=args.data_dir,
        batch_size=args.batch_size,
    )


Overwriting src/evaluate.py


In [28]:
#CELDA 12 - evaluación
%cd /content/IA-Image-Detection

!python src/evaluate.py \
    --model_path "models/saved_models/best_model.pth" \
    --data_dir "data/raw/ALL_full" \
    --batch_size 128


/content/IA-Image-Detection
🖥️ Usando dispositivo: cuda
Cargando datos desde los siguientes roots:
 - data/raw/ALL_full
Clases detectadas: ['ai', 'nature']
Nº de datasets combinados: 1
Checkpoint cargado desde models/saved_models/best_model.pth

📊 === CLASIFICATION REPORT ===

              precision    recall  f1-score   support

          ai     0.9989    0.9988    0.9989     12000
      nature     0.9988    0.9989    0.9989     12000

    accuracy                         0.9989     24000
   macro avg     0.9989    0.9989    0.9989     24000
weighted avg     0.9989    0.9989    0.9989     24000


📉 === MATRIZ DE CONFUSIÓN ===

[[11986    14]
 [   13 11987]]


In [29]:
#CELDA 13 - inferencias de ejemplo
%cd /content/IA-Image-Detection

!python src/infer.py \
    --image_path "data/raw/generador2_BigGAN_descomprimido/train/ai/999_biggan_00198.png" \
    --model_path "models/saved_models/best_model.pth"


/content/IA-Image-Detection
🖥️  Usando dispositivo: cuda
Checkpoint cargado desde models/saved_models/best_model.pth

📌 Imagen: data/raw/generador2_BigGAN_descomprimido/train/ai/999_biggan_00198.png
📦 Modelo: models/saved_models/best_model.pth
------------------------------
🔍 Predicción: Generada
📊 Confianza: 99.90%



In [37]:
#CELDA 14 - gradcam de ejemplo
%cd /content/IA-Image-Detection

!python src/gradcam_infer.py \
    --image_path "data/raw/generador2_BigGAN_descomprimido/train/ai/999_biggan_00186.png" \
    --model_path "models/saved_models/best_model.pth" \
    --target_layer "stages.2.blocks.4.conv_dw"


/content/IA-Image-Detection
🖥️ Usando dispositivo: cuda
Checkpoint cargado desde models/saved_models/best_model.pth
[ WARN:0@8.289] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
🔥 Grad-CAM guardado en: results/gradcams/999_biggan_00186_gradcam.jpg

========= RESULTADO =========
📌 Imagen: data/raw/generador2_BigGAN_descomprimido/train/ai/999_biggan_00186.png
📦 Modelo: models/saved_models/best_model.pth
🎯 Capa usada: stages.2.blocks.4.conv_dw
🔍 Predicción: Generada
💾 Guardado en: results/gradcams/999_biggan_00186_gradcam.jpg

